In [ ]:
# Import modules
import matplotlib.pyplot as plt 
import pandas as pd
import yaml

def read_dict(file):
    with open(file,'r') as f:
        return yaml.safe_load(f)

# Import data
alpha = snakemake.params.alpha

metrics = []
index = []
for file in snakemake.input:
    metrics.append(read_dict(file))
    index.append(file.split('/')[-2])

metrics = pd.DataFrame(metrics,index=index)
metrics['dd_2/1'] = metrics['dd_m2'] / metrics['dd_m1']



In [ ]:
# Plot reduction metrics
fig, axes = plt.subplots(2,1,figsize=(9,6),dpi=300)

def fraction(metric):
    '''Calculates removed fraction for some metric
    '''
    return (metrics[metric].loc['giant'] - metrics[metric].loc[metrics.index != 'giant']) / metrics[metric].loc['giant']

axes[0].plot(alpha.values(),
             fraction('n_nodes'),
             '--o'
)
axes[0].set_ylabel("Fraction of nodes removed")
axes[0].set_xticks([])

axes[1].plot(alpha.values(),
             fraction('n_edges'),
             '--o'
)
axes[1].set_ylabel("Fraction of edges removed")
axes[1].set_xlabel("Reduction")

# Title
n_nodes = metrics.loc['giant','n_nodes']
n_edges = metrics.loc['giant','n_edges']
axes[0].set_title(f"{snakemake.wildcards.graph.capitalize()} graph: |V|={n_nodes:.2e}, |E|={n_edges:.2e}")
plt.savefig(snakemake.output[0])

In [ ]:
def plot_metric(metric,ax):
    ax.axhline(metrics.loc['giant',metric],linestyle=':',color='red')
    ax.plot(alpha.values(),metrics.loc[metrics.index != 'giant', metric], '--o')

fig, axes = plt.subplots(5,1,figsize=(8,15),dpi=300)

for i, metric in enumerate(['n_components','connectivity','density','clustering','assortativity']):
    plot_metric(metric,axes[i])
    
    axes[i].set_ylabel(metric.capitalize())

    if metric != 'assortativity':
        axes[i].set_xticks([])
        axes[i].set_ylim([0,1.1])
    else:
        axes[i].set_ylim([-1.1,1.1])

    if i == 0:
        axes[i].set_title(f"Graph metrics")
        axes[i].legend(["Giant","Miniatures"])

plt.savefig(snakemake.output[1])

In [ ]:
fig, axes = plt.subplots(5,1,figsize=(8,15),dpi=300)

names = [
    "eig_1",
    "eig_2",
    "dd_m1",
    "dd_m2",
    "dd_2/1"
]

for i, metric in enumerate(names):
    plot_metric(metric,axes[i])
    axes[i].set_ylabel(metric.capitalize())

    if i == 4:
        axes[i].set_xlabel("Reduction")

plt.savefig(snakemake.output[2])